In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

2023-11-13 12:32:17.009738: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-13 12:32:17.011032: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-13 12:32:17.036852: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-13 12:32:17.037398: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-13 12:32:17.603007: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [10]:
dataset,dataset_info = tfds.load('malaria',with_info=True,as_supervised=True,shuffle_files=True,split=['train'])

In [14]:
for data in dataset[0].take(4):
    print(data)

(<tf.Tensor: shape=(151, 115, 3), dtype=uint8, numpy=
array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)>, <tf.Tensor: shape=(), dtype=int64, numpy=1>)
(<tf.Tensor: shape=(121, 115, 3), dtype=uint8, numpy=
array([[[0, 0, 0],
        [0, 0, 0],
       

In [58]:
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

def create_split(dataset,TRAIN_RATIO,VAL_RATIO,TEST_RATIO):
    DATASET_SIZE = len(dataset)
    train_data = dataset.take(int(TRAIN_RATIO*DATASET_SIZE))
    val_dataset = dataset.skip(int(TRAIN_RATIO*DATASET_SIZE))
    val_data = val_dataset.take(int(VAL_RATIO*DATASET_SIZE))
    test_data = val_dataset.skip(int(TEST_RATIO*DATASET_SIZE))
    return train_data,val_data,test_data

train_data,val_data,test_data = create_split(dataset[0],TRAIN_RATIO,VAL_RATIO,TEST_RATIO)


In [59]:
print(list(train_data.take(1).as_numpy_iterator()))

[(array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8), 0)]


In [60]:
IMG_SIZE = 224
def resize_rescale(image,label):
    return tf.image.resize(image,(IMG_SIZE,IMG_SIZE))/255.0,label


train_data = train_data.map(resize_rescale)
val_data = val_data.map(resize_rescale)
test_data = test_data.map(resize_rescale)

In [61]:
train_data = train_data.shuffle(buffer_size=8,reshuffle_each_iteration=True).batch(32).prefetch(tf.data.AUTOTUNE)
val_data = val_data.shuffle(buffer_size=8,reshuffle_each_iteration=True).batch(32).prefetch(tf.data.AUTOTUNE)

In [62]:
from tensorflow.keras.layers import Normalization,Dense,InputLayer,BatchNormalization,MaxPool2D,Flatten,Conv2D

model = tf.keras.Sequential()
model.add(InputLayer(input_shape=(224,224,3)))
model.add(Conv2D(filters=6,kernel_size=3,strides=1,padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2,strides=1))
model.add(Conv2D(filters=16,kernel_size=3,strides=1,padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=2,strides=1))
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(10,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1))

In [63]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 222, 222, 6)       168       
                                                                 
 batch_normalization_23 (Ba  (None, 222, 222, 6)       24        
 tchNormalization)                                               
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 221, 221, 6)       0         
 ng2D)                                                           
                                                                 
 conv2d_15 (Conv2D)          (None, 219, 219, 16)      880       
                                                                 
 batch_normalization_24 (Ba  (None, 219, 219, 16)      64        
 tchNormalization)                                               
                                                     

In [64]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [65]:
model.compile(optimizer=Adam(),loss=BinaryCrossentropy(),metrics='accuracy')


In [66]:
history =model.fit(train_data,validation_data=val_data,epochs=10,verbose=True)

Epoch 1/10
 42/689 [>.............................] - ETA: 9:13 - loss: 5.9158 - accuracy: 0.4821

KeyboardInterrupt: 

In [68]:
model.evaluate(test_data.batch(32))

87/87 [==============================] - 12s 119ms/step - loss: 0.6967 - accuracy: 0.5031


[0.6966650485992432, 0.5030830502510071]